# Air Quality Data Cleaning and Imputation Pipeline

Steps:
1. Drop columns with >70% missing values
2. City-wise statistical fill (median per city)
3. Regression-based imputation (order matters)
4. MICE or KNN for remaining missing values
5. Recalculate AQI_Bucket

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# STEP 1: DROP USELESS COLUMNS (>70% missing)

In [2]:
def step1_drop_high_missing_columns(df, threshold=70):
    """Drop columns with missing percentage > threshold"""
    print("=" * 80)
    print("STEP 1: Dropping columns with >{}% missing values".format(threshold))
    print("=" * 80)
    
    missing_pct = (df.isnull().sum() / len(df) * 100)
    cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()
    
    # Don't drop City or AQI_Bucket
    cols_to_drop = [col for col in cols_to_drop if col not in ['City', 'AQI_Bucket']]
    
    print(f"\nColumns to drop (>{threshold}% missing):")
    for col in cols_to_drop:
        print(f"  - {col}: {missing_pct[col]:.2f}% missing")
    
    df_clean = df.drop(columns=cols_to_drop)
    print(f"\nDropped {len(cols_to_drop)} columns")
    print(f"Remaining columns: {df_clean.shape[1]}")
    
    return df_clean, cols_to_drop

# STEP 2: CITY-WISE STATISTICAL FILL (MEDIAN)

In [3]:
def step2_city_wise_median_fill(df):
    """Fill missing values with city-wise median"""
    print("\n" + "=" * 80)
    print("STEP 2: City-wise median imputation")
    print("=" * 80)
    
    df_filled = df.copy()
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove AQI and AQI_Bucket from this step (will handle later)
    numeric_cols = [col for col in numeric_cols if col not in ['AQI']]
    
    filled_counts = {}
    
    for col in numeric_cols:
        if df[col].isnull().any():
            before_missing = df[col].isnull().sum()
            
            # Calculate city-wise median and fill
            city_medians = df.groupby('City')[col].transform('median')
            df_filled[col] = df_filled[col].fillna(city_medians)
            
            after_missing = df_filled[col].isnull().sum()
            filled = before_missing - after_missing
            filled_counts[col] = filled
            
            print(f"  {col}: Filled {filled} values with city-wise median")
    
    return df_filled

# STEP 3: REGRESSION-BASED IMPUTATION 

In [4]:
def regression_impute(df, target_col, predictor_cols, min_samples=50):
    """
    Impute target_col using linear regression with predictor_cols
    """
    df_temp = df.copy()
    
    # Only use rows where target is missing but predictors are available
    missing_mask = df_temp[target_col].isnull()
    
    # Training data: rows where target AND predictors are non-null
    train_mask = ~df_temp[target_col].isnull()
    for pred in predictor_cols:
        train_mask = train_mask & ~df_temp[pred].isnull()
    
    # Prediction data: rows where target is null but predictors are non-null
    predict_mask = missing_mask.copy()
    for pred in predictor_cols:
        predict_mask = predict_mask & ~df_temp[pred].isnull()
    
    if train_mask.sum() < min_samples or predict_mask.sum() == 0:
        return df_temp, 0
    
    # Train model
    X_train = df_temp.loc[train_mask, predictor_cols]
    y_train = df_temp.loc[train_mask, target_col]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict missing values
    X_predict = df_temp.loc[predict_mask, predictor_cols]
    predictions = model.predict(X_predict)
    
    # Fill predictions (ensure non-negative for pollutant values)
    predictions = np.maximum(predictions, 0)
    df_temp.loc[predict_mask, target_col] = predictions
    
    return df_temp, predict_mask.sum()


def step3_regression_imputation(df):
    """Regression-based imputation in specific order"""
    print("\n" + "=" * 80)
    print("STEP 3: Regression-based imputation (ORDER MATTERS)")
    print("=" * 80)
    
    df_reg = df.copy()
    
    # 3.1: NO, NO2 → NOx
    print("\n3.1: Imputing NOx from NO and NO2...")
    if 'NOx' in df_reg.columns and 'NO' in df_reg.columns and 'NO2' in df_reg.columns:
        df_reg, filled = regression_impute(df_reg, 'NOx', ['NO', 'NO2'])
        print(f"    Filled {filled} NOx values")
    
    # 3.2: PM2.5 ↔ PM10 (bidirectional)
    print("\n3.2: Imputing PM2.5 and PM10 (bidirectional)...")
    if 'PM2.5' in df_reg.columns and 'PM10' in df_reg.columns:
        # PM2.5 from PM10
        df_reg, filled1 = regression_impute(df_reg, 'PM2.5', ['PM10'])
        print(f"    Filled {filled1} PM2.5 values from PM10")
        
        # PM10 from PM2.5
        df_reg, filled2 = regression_impute(df_reg, 'PM10', ['PM2.5'])
        print(f"    Filled {filled2} PM10 values from PM2.5")
    
    # 3.3: Gaseous pollutants (CO, SO2, O3)
    print("\n3.3: Imputing gaseous pollutants (CO, SO2, O3)...")
    
    # Build predictor list from available pollutants
    available_predictors = [col for col in ['NO', 'NO2', 'NOx', 'PM2.5', 'PM10'] 
                           if col in df_reg.columns]
    
    for target in ['CO', 'SO2', 'O3']:
        if target in df_reg.columns and len(available_predictors) > 0:
            df_reg, filled = regression_impute(df_reg, target, available_predictors)
            print(f"    Filled {filled} {target} values")
    
    # 3.4: AQI (last)
    print("\n3.4: Imputing AQI from all available pollutants...")
    if 'AQI' in df_reg.columns:
        aqi_predictors = [col for col in ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 
                                          'CO', 'SO2', 'O3'] 
                         if col in df_reg.columns]
        if len(aqi_predictors) > 0:
            df_reg, filled = regression_impute(df_reg, 'AQI', aqi_predictors)
            print(f"    Filled {filled} AQI values")
    
    return df_reg

# STEP 4: MICE OR KNN FOR LEFTOVERS

In [5]:
def step4_mice_knn_imputation(df, method='mice'):
    """Apply MICE or KNN imputation for remaining missing values"""
    print("\n" + "=" * 80)
    print(f"STEP 4: {method.upper()} imputation for remaining missing values")
    print("=" * 80)
    
    df_final = df.copy()
    
    # Get numeric columns (exclude AQI for now)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in ['AQI']]
    
    # Check remaining missing values
    missing_before = df[numeric_cols].isnull().sum()
    print("\nMissing values before MICE/KNN:")
    for col in numeric_cols:
        if missing_before[col] > 0:
            print(f"  {col}: {missing_before[col]} ({missing_before[col]/len(df)*100:.2f}%)")
    
    if missing_before.sum() == 0:
        print("\nNo missing values to impute!")
        return df_final
    
    # Apply imputation
    if method.lower() == 'mice':
        print(f"\nApplying MICE imputation (max_iter=10)...")
        imputer = IterativeImputer(max_iter=10, random_state=42, verbose=0)
    else:  # KNN
        print(f"\nApplying KNN imputation (n_neighbors=5)...")
        imputer = KNNImputer(n_neighbors=5)
    
    df_final[numeric_cols] = imputer.fit_transform(df_final[numeric_cols])
    
    # Ensure non-negative values
    for col in numeric_cols:
        df_final[col] = np.maximum(df_final[col], 0)
    
    missing_after = df_final[numeric_cols].isnull().sum()
    print("\nMissing values after MICE/KNN:")
    print(f"  Total: {missing_after.sum()}")
    
    return df_final

# STEP 5: RECALCULATE AQI_BUCKET

In [6]:
def calculate_aqi_bucket(aqi_value):
    """Calculate AQI bucket based on AQI value"""
    if pd.isnull(aqi_value):
        return None
    elif aqi_value <= 50:
        return 'Good'
    elif aqi_value <= 100:
        return 'Satisfactory'
    elif aqi_value <= 200:
        return 'Moderate'
    elif aqi_value <= 300:
        return 'Poor'
    elif aqi_value <= 400:
        return 'Very Poor'
    else:
        return 'Severe'


def step5_recalculate_aqi_bucket(df):
    """Recalculate AQI_Bucket based on AQI values"""
    print("\n" + "=" * 80)
    print("STEP 5: Recalculating AQI_Bucket")
    print("=" * 80)
    
    df_final = df.copy()
    
    if 'AQI' in df_final.columns:
        # Fill remaining AQI missing values if any
        if df_final['AQI'].isnull().any():
            print(f"\nFilling {df_final['AQI'].isnull().sum()} remaining AQI values with overall median...")
            df_final['AQI'].fillna(df_final['AQI'].median(), inplace=True)
        
        # Recalculate AQI_Bucket
        print("\nRecalculating AQI_Bucket based on AQI values...")
        df_final['AQI_Bucket'] = df_final['AQI'].apply(calculate_aqi_bucket)
        
        # Show distribution
        print("\nAQI_Bucket distribution:")
        print(df_final['AQI_Bucket'].value_counts().sort_index())
    
    return df_final

# MAIN PIPELINE

In [7]:
def run_cleaning_pipeline(input_file, output_file, drop_threshold=70, imputation_method='mice'):
    """
    Run complete data cleaning pipeline
    
    Parameters:
    -----------
    input_file : str
        Path to input CSV file
    output_file : str
        Path to save cleaned CSV file
    drop_threshold : int
        Percentage threshold for dropping columns (default: 70)
    imputation_method : str
        'mice' or 'knn' for final imputation (default: 'mice')
    """
    print("\n" + "=" * 80)
    print("AIR QUALITY DATA CLEANING PIPELINE")
    print("=" * 80)
    
    # Load data
    print(f"\nLoading data from: {input_file}")
    df = pd.read_csv(input_file)
    print(f"Initial shape: {df.shape}")
    print(f"Initial missing values: {df.isnull().sum().sum()}")
    
    # Step 1: Drop high-missing columns
    df_clean, dropped_cols = step1_drop_high_missing_columns(df, threshold=drop_threshold)
    
    # Step 2: City-wise median fill
    df_clean = step2_city_wise_median_fill(df_clean)
    
    # Step 3: Regression-based imputation
    df_clean = step3_regression_imputation(df_clean)
    
    # Step 4: MICE/KNN imputation
    df_clean = step4_mice_knn_imputation(df_clean, method=imputation_method)
    
    # Step 5: Recalculate AQI_Bucket
    df_clean = step5_recalculate_aqi_bucket(df_clean)
    
    # Final summary
    print("\n" + "=" * 80)
    print("FINAL SUMMARY")
    print("=" * 80)
    print(f"\nFinal shape: {df_clean.shape}")
    print(f"Dropped columns: {dropped_cols}")
    print(f"\nRemaining missing values:")
    missing_final = df_clean.isnull().sum()
    print(missing_final[missing_final > 0])
    if missing_final.sum() == 0:
        print("  None! ✓")
    
    # Save cleaned data
    print(f"\nSaving cleaned data to: {output_file}")
    df_clean.to_csv(output_file, index=False)
    print("Done!")
    
    return df_clean

# EXECUTION

In [8]:
if __name__ == "__main__":
    # Configuration
    INPUT_FILE = '/kaggle/input/city-day-data/city_day.csv'
    OUTPUT_FILE = '/kaggle/working/city_day_cleaned.csv'
    DROP_THRESHOLD = 70  # Drop columns with >70% missing
    IMPUTATION_METHOD = 'mice'  # or 'knn'
    
    # Run pipeline
    df_cleaned = run_cleaning_pipeline(
        input_file=INPUT_FILE,
        output_file=OUTPUT_FILE,
        drop_threshold=DROP_THRESHOLD,
        imputation_method=IMPUTATION_METHOD
    )
    
    # Show sample of cleaned data
    print("\n" + "=" * 80)
    print("SAMPLE OF CLEANED DATA")
    print("=" * 80)
    print(df_cleaned.head(10))
    
    # Show statistics
    print("\n" + "=" * 80)
    print("DESCRIPTIVE STATISTICS")
    print("=" * 80)
    print(df_cleaned.describe())


AIR QUALITY DATA CLEANING PIPELINE

Loading data from: /kaggle/input/city-day-data/city_day.csv
Initial shape: (27284, 15)
Initial missing values: 60173
STEP 1: Dropping columns with >70% missing values

Columns to drop (>70% missing):

Dropped 0 columns
Remaining columns: 15

STEP 2: City-wise median imputation
  PM2.5: Filled 2351 values with city-wise median
  PM10: Filled 6903 values with city-wise median
  NO: Filled 1336 values with city-wise median
  NO2: Filled 1339 values with city-wise median
  NOx: Filled 1225 values with city-wise median
  NH3: Filled 6649 values with city-wise median
  CO: Filled 576 values with city-wise median
  SO2: Filled 1608 values with city-wise median
  O3: Filled 1617 values with city-wise median
  Benzene: Filled 1633 values with city-wise median
  Toluene: Filled 2914 values with city-wise median
  Xylene: Filled 4144 values with city-wise median

STEP 3: Regression-based imputation (ORDER MATTERS)

3.1: Imputing NOx from NO and NO2...
    Fill